# DysonianLineCNN - Нейронна мережа для передбачення параметрів лінії Дайсона

Цей notebook містить код для навчання CNN моделі для передбачення параметрів лінії Дайсона.

## Структура проекту:
- `colab_gpu_check.py` - перевірка системних ресурсів
- `data_loader.py` - завантаження та підготовка даних
- `model.py` - архітектура нейронної мережі
- `trainer.py` - навчання моделі
- `utils.py` - допоміжні функції
- `model_visualizer.py` - візуалізація моделі

In [ ]:
# ⚡️ Клонування репозиторію для запуску в Colab
!git clone https://github.com/AndriiUriadov/DysonLineParamsExtractorCNN.git
%cd DysonLineParamsExtractorCNN

In [ ]:
# Виконуємо код перевірки середовища виконання
import subprocess
import sys
# Виконуємо файл як модуль
exec(open('colab_gpu_check.py').read())

In [ ]:
# Завантаження та попередня обробка даних
from data_loader import load_and_preprocess

# Завантажуємо та обробляємо дані
X_train, X_val, X_test, y_train, y_val, y_test, y_train_dict, y_val_dict, y_test_dict, scaler_y, y_min, y_max = load_and_preprocess()

print("✅ Дані успішно завантажено та оброблено!")
print(f"   Train samples: {X_train.shape[0]}")
print(f"   Val samples: {X_val.shape[0]}")
print(f"   Test samples: {X_test.shape[0]}")

In [ ]:
# Створення моделі
from model import create_model
import torch

# Визначаємо пристрій
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🎯 Використовуємо пристрій: {device}")

# Створюємо модель
model = create_model(
    input_size=4096,
    hidden_sizes=[512, 256, 128],
    num_heads=8,
    dropout=0.2,
    device=device
)

print("✅ Модель створена!")

In [ ]:
# Створення тренера
from trainer import create_trainer

# Оптимізації пам'яті GPU
import torch
torch.cuda.empty_cache()
torch.backends.cudnn.benchmark = True

# Створюємо тренер
trainer = create_trainer(
    model=model,
    learning_rate=0.001,
    weight_decay=1e-5,
    device=device
)

# Створюємо DataLoader
train_loader, val_loader, test_loader = trainer.create_data_loaders(
    X_train, X_val, X_test, y_train, y_val, y_test, batch_size=16
)

print("✅ Тренер готовий до навчання!")

In [ ]:
# Навчання моделі
print("🚀 Початок навчання...")

# Навчаємо модель
history = trainer.train(
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=100,
    early_stopping_patience=20
)

print("✅ Навчання завершено!")

In [ ]:
# Оцінка моделі
print("📊 Оцінка моделі...")

# Оцінюємо на тестових даних
test_metrics = trainer.evaluate(test_loader)

print("✅ Оцінка завершена!")

In [ ]:
# Візуалізація результатів
from utils import plot_training_results, plot_predictions_vs_actual
from model_visualizer import visualize_model_architecture

# Візуалізуємо архітектуру моделі
visualize_model_architecture(model, device)

# Візуалізуємо результати навчання
plot_training_results(history)

# Візуалізуємо передбачення
plot_predictions_vs_actual(model, test_loader, device, scaler_y)

print("✅ Візуалізація завершена!")